# Initial implementation of PIICM

Will make a stab at implementing the original version of PIICM in the newest version of gpytorch. There are a few ingredients that are needed

1. The kernel  -- we need to construct a permutation invariant covariance function.
    1. Needs to return objects with custom .matmul routines, custom log_det calculations etc. This was done in the original implementation by extending the `LazyTensor` class, which is no longer available. Will need to work with the `LinearOperators` instead, but hopefully should be rather straight-forward.
2. The likelihood -- we need a multi-task normal likelihood that allows us to add a observation specific noise term
3. Model -- putting these two together and maximizing the marginal likelihood using CG for the inverse solve and a custom log-determinant calculation involving Weyl's inequality.

It needs to all play nice with the current classes, extending were needed but otherwise not interferring with gpytorch.

## Framework of LinearOperators

Have constructed three new classes, extending the `LinearOperator` class.
 1. `GPattKroneckerProductLinearOperator` simple extension of `KroneckerProductLinearOperator`
 2. `GPattKroneckerSumLinearOperator`, the result of `GPattKroneckerProductLinearOperator`+`GPattKroneckerProductLinearOperator`, essentially the prior of the PIICM
 3. `GPaddKroneckerSumAddedDiagLinearOperator`, the result of `GPattKroneckerSumLinearOperator`+`DiagLinearOperator` -- for when the likelihood is added
 
Can verify that these do what they are supposed to below 

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import torch
from linear_operator.operators import DenseLinearOperator, KroneckerProductLinearOperator, DiagLinearOperator
from synpred.linear_operator.operators.gpatt_kronecker_product_linear_operator import GPattKroneckerProductLinearOperator

# Generate two tensors
a = torch.tensor([[4, 0, 2], [0, 3, -1], [2, -1, 3]], dtype=torch.float)
b = torch.tensor([[2, 1], [1, 2]], dtype=torch.float)

# The Kronecker product
c = KroneckerProductLinearOperator(DenseLinearOperator(a),DenseLinearOperator(b))

# Wrap it as a GPattKroneckerProduct
d = GPattKroneckerProductLinearOperator(c)
print(d)

# Summing two GPattKroneckerProducts
e = d + d
print(e)

# Adding a diagonal
diag = DiagLinearOperator(e.diagonal())
f = e + diag
print(f)


In [9]:
f._logdet()

computing log-det with Weyl's inequality!


AttributeError: 'GPattKroneckerSumAddedDiagLinearOperator' object has no attribute '_lazy_tensor'

In [ ]:
e.evaluate_kernel().numpy()

In [ ]:
from synpred.linear_operator.operators.gpatt_kronecker_product_linear_operator import *

In [ ]:
GPattKroneckerProductLinearOperator